In [31]:
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from scipy.optimize import curve_fit
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
import warnings

In [41]:
warnings.filterwarnings('ignore')

In [34]:
DATA_PATH = '/kaggle/input/mallorn-dataset' 
print(f"Data Path: {DATA_PATH}")

Data Path: /kaggle/input/mallorn-dataset


In [35]:
def bazin_func(t, A, B, t0, tau_fall, tau_rise):
    with np.errstate(over='ignore', invalid='ignore'):
        flux = A * (np.exp(-(t - t0) / tau_fall) / (1 + np.exp(-(t - t0) / tau_rise))) + B
    return np.nan_to_num(flux)

In [36]:
def fit_bazin(time, flux, flux_err):
    if len(time) < 5: 
        return {k: np.nan for k in ['A', 'B', 't0', 'tau_fall', 'tau_rise', 'chi2']}

    peak_idx = np.argmax(flux)
    # Initial guesses
    p0 = [flux[peak_idx], np.min(flux), time[peak_idx], 50.0, 10.0]
    # Bounds
    bounds = ([0, -np.inf, time.min()-50, 1e-3, 1e-3], [np.inf, np.inf, time.max()+50, 500, 500])

    try:
        popt, _ = curve_fit(bazin_func, time, flux, p0=p0, sigma=flux_err, bounds=bounds, maxfev=1000)
        residuals = flux - bazin_func(time, *popt)
        chi2 = np.sum((residuals / flux_err)**2) / (len(time) - 5)
        return {'A': popt[0], 'B': popt[1], 't0': popt[2], 'tau_fall': popt[3], 'tau_rise': popt[4], 'chi2': chi2}
    except:
        return {k: np.nan for k in ['A', 'B', 't0', 'tau_fall', 'tau_rise', 'chi2']}

In [37]:
def get_gp_prediction(time, flux, flux_err, t_query):
    if len(time) < 3: return np.nan
    kernel = C(1.0) * RBF(length_scale=20.0) + WhiteKernel(noise_level=1.0)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=flux_err**2, n_restarts_optimizer=0)
    try:
        gp.fit(time.reshape(-1, 1), flux)
        pred, _ = gp.predict(np.array([[t_query]]), return_std=True)
        return pred[0]
    except:
        return np.nan

In [38]:
def process_single_object(obj_id, df_obj):
    feats = {'object_id': obj_id}
    
    t_min = df_obj['Time (MJD)'].min()
    df_obj['Time_Rel'] = df_obj['Time (MJD)'] - t_min
    
    filters = ['u', 'g', 'r', 'i', 'z', 'y']
    
    peak_time = np.nan
    max_flux_global = -np.inf
    
    for f in filters:
        df_f = df_obj[df_obj['Filter'] == f]
        if df_f.empty:
            for stat in ['mean', 'max', 'min', 'std', 'skew']:
                feats[f'{f}_{stat}'] = np.nan
            continue
            
        feats[f'{f}_mean'] = df_f['Flux'].mean()
        feats[f'{f}_max'] = df_f['Flux'].max()
        feats[f'{f}_min'] = df_f['Flux'].min()
        feats[f'{f}_std'] = df_f['Flux'].std()
        feats[f'{f}_skew'] = df_f['Flux'].skew()
        
        if f in ['g', 'r']:
            current_max = df_f['Flux'].max()
            if current_max > max_flux_global:
                max_flux_global = current_max
                peak_time = df_f.loc[df_f['Flux'].idxmax(), 'Time_Rel']

        if f in ['g', 'r', 'i']:
            bazin = fit_bazin(df_f['Time_Rel'].values, df_f['Flux'].values, df_f['Flux_err'].values)
            for k, v in bazin.items():
                feats[f'bazin_{f}_{k}'] = v

    if not np.isnan(peak_time):
        flux_at_peak = {}
        for f in filters:
            df_f = df_obj[df_obj['Filter'] == f]
            flux_at_peak[f] = get_gp_prediction(
                df_f['Time_Rel'].values, df_f['Flux'].values, df_f['Flux_err'].values, peak_time
            )
        
        pairs = [('u','g'), ('g','r'), ('r','i'), ('i','z')]
        for f1, f2 in pairs:
            val1 = flux_at_peak.get(f1, np.nan)
            val2 = flux_at_peak.get(f2, np.nan)
            feats[f'gp_color_{f1}_{f2}'] = val1 - val2 if (not np.isnan(val1) and not np.isnan(val2)) else np.nan
            
    return feats

In [39]:
def extract_features_parallel(log_df, data_path, n_jobs=-1):
    """Load từng file lightcurve và trích xuất features song song"""
    print("Loading raw lightcurves into memory...")
    all_chunks = []
    unique_splits = log_df['split'].unique()
    
    for split in unique_splits:
        is_train = 'target' in log_df.columns
        filename = 'train_full_lightcurves.csv' if is_train else 'test_full_lightcurves.csv'
        
        path = os.path.join(data_path, split, filename)
        if os.path.exists(path):
            df_chunk = pd.read_csv(path)
            valid_ids = set(log_df[log_df['split'] == split]['object_id'])
            df_chunk = df_chunk[df_chunk['object_id'].isin(valid_ids)]
            all_chunks.append(df_chunk)
            
    if not all_chunks:
        print("Error: No lightcurves found!")
        return pd.DataFrame()

    full_lc = pd.concat(all_chunks)
    
    grouped = full_lc.groupby('object_id')
    object_ids = list(grouped.groups.keys())
    
    print(f"Extracting features for {len(object_ids)} objects using {n_jobs} cores...")
    
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(process_single_object)(obj_id, grouped.get_group(obj_id))
        for obj_id in tqdm(object_ids)
    )
    
    return pd.DataFrame(results)

from tqdm.auto import tqdm

In [47]:
print("--- PROCESSING TRAIN DATA ---")
train_log = pd.read_csv(os.path.join(DATA_PATH, 'train_log.csv'))

df_train_features = extract_features_parallel(train_log, DATA_PATH, n_jobs=4)

df_train_final = train_log.merge(df_train_features, on='object_id', how='left')
print(f"Train Data Shape: {df_train_final.shape}")

--- PROCESSING TRAIN DATA ---
Loading raw lightcurves into memory...
Extracting features for 3043 objects using 4 cores...


  0%|          | 0/3043 [00:00<?, ?it/s]

Train Data Shape: (3043, 60)


In [48]:
ignore_cols = ['object_id', 'target', 'split', 'English Translation', 'SpecType']
features = [c for c in df_train_final.columns if c not in ignore_cols]

In [50]:
X = df_train_final[features]
y = df_train_final['target']

In [51]:
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

In [52]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.03,
    'num_leaves': 31,
    'max_depth': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'n_estimators': 2000,
    'verbose': -1,
    'n_jobs': 4
}

In [53]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models = []
thresholds = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()
    
    clf = lgb.LGBMClassifier(**params, scale_pos_weight=pos_weight)
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            callbacks=[lgb.early_stopping(100, verbose=False)])
    
    probs = clf.predict_proba(X_val)[:, 1]
    best_f1 = 0
    best_th = 0.5
    for th in np.linspace(0.1, 0.9, 50):
        score = f1_score(y_val, (probs > th).astype(int))
        if score > best_f1:
            best_f1 = score
            best_th = th
            
    print(f"Fold {fold+1} F1: {best_f1:.4f} (Threshold: {best_th:.2f})")
    models.append(clf)
    thresholds.append(best_th)

avg_threshold = np.mean(thresholds)
print(f"Average Optimal Threshold: {avg_threshold:.2f}")

Fold 1 F1: 0.5143 (Threshold: 0.18)
Fold 2 F1: 0.6667 (Threshold: 0.26)
Fold 3 F1: 0.6769 (Threshold: 0.25)
Fold 4 F1: 0.5778 (Threshold: 0.66)
Fold 5 F1: 0.5484 (Threshold: 0.38)
Average Optimal Threshold: 0.34


In [54]:
test_log = pd.read_csv(os.path.join(DATA_PATH, 'test_log.csv'))

df_test_features = extract_features_parallel(test_log, DATA_PATH, n_jobs=4)

df_test_final = test_log.merge(df_test_features, on='object_id', how='left')

Loading raw lightcurves into memory...
Extracting features for 7135 objects using 4 cores...


  0%|          | 0/7135 [00:00<?, ?it/s]

In [55]:
X_test = df_test_final[features]
for col in X_test.select_dtypes(include=['object']).columns:
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce').fillna(0)

In [56]:
test_probs = np.zeros(len(X_test))
for model in models:
    test_probs += model.predict_proba(X_test)[:, 1] / len(models)

In [57]:
predictions = (test_probs > avg_threshold).astype(int)

In [58]:
submission = pd.DataFrame({
    'object_id': df_test_final['object_id'],
    'prediction': predictions
})

submission.to_csv('submission.csv', index=False)
print("\nSuccess! Saved submission.csv")
print(submission.head())


Success! Saved submission.csv
                      object_id  prediction
0      Eluwaith_Mithrim_nothrim           0
1            Eru_heledir_archam           0
2             Gonhir_anann_fuin           0
3  Gwathuirim_haradrim_tegilbor           0
4              achas_minai_maen           0
